This project selects the 10/20 day rotation strategy from 2021-11-03 to 2023-09-13 CSI 300 and CSI 500 index downloaded by Yingwei Finance for backtesting, without considering fixed income assets, just to test the return rate of only one asset, stocks. When both large and small stocks stop trading, the data of fixed income yield to maturity is also difficult to predict, and it is inconvenient to join the evaluation indicators

In [52]:
import warnings
warnings.filterwarnings('ignore')

In [20]:
import pandas as pd

# 1. read csv
hs300 = pd.read_csv('沪深300.csv')
zz500 = pd.read_csv('中证500.csv')

# 2. manage date style
hs300['Date'] = pd.to_datetime(hs300['Date'], format='%m/%d/%y')
zz500['Date'] = pd.to_datetime(zz500['Date'], format='%m/%d/%y')

In [50]:
hs300.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2021-11-03,4835.810059,4858.709961,4804.370117,4821.109863,4821.109863,142800
1,2021-11-04,4841.089844,4873.939941,4834.740234,4868.740234,4868.740234,132600
2,2021-11-05,4858.899902,4889.680176,4841.740234,4842.350098,4842.350098,147800
3,2021-11-08,4841.149902,4866.080078,4827.689941,4848.180176,4848.180176,137400
4,2021-11-09,4858.250000,4870.790039,4818.540039,4846.740234,4846.740234,118400


In [51]:
zz500.head()

,Date,Open,High,Low,Close,volume
0,2021-11-03,6975.06,7034.09,6962.87,7017.85,15081.46
1,2021-11-04,7031.95,7073.85,7018.61,7073.85,15265.20
2,2021-11-05,7071.08,7073.85,6991.15,6991.91,17724.14
3,2021-11-08,6986.50,7034.14,6984.81,7026.97,14636.09
4,2021-11-09,7037.57,7087.02,7028.31,7081.75,14169.68


In [55]:
import pandas as pd

class RotationStrategy:
    def __init__(self, hs300_path, zz500_path):
        self.hs300 = pd.read_csv(hs300_path)
        self.zz500 = pd.read_csv(zz500_path)
        
        #time
        self.hs300['Date'] = pd.to_datetime(self.hs300['Date'], format='%m/%d/%y')
        self.zz500['Date'] = pd.to_datetime(self.zz500['Date'],  format='%m/%d/%y')
        
        # merge
        self.data = pd.merge(self.hs300[['Date', 'Close']], 
                             self.zz500[['Date', 'Close']], 
                             on='Date', how='inner', 
                             suffixes=('_hs300', '_zz500'))
        
    def execute_strategy(self, period=20):
        # return
        self.data['return_hs300'] = self.data['Close_hs300'].pct_change(period)
        self.data['return_zz500'] = self.data['Close_zz500'].pct_change(period)
        
        # moving
        self.data['position'] = (self.data['return_hs300'] > self.data['return_zz500']).shift(1).astype(float)
        self.data['daily_return'] = self.data['position'] * self.data['return_hs300'] + (1 - self.data['position']) * self.data['return_zz500']
        
        # Calculate strategy net value
        self.data['strategy'] = (1 + self.data['daily_return']).cumprod()

    def get_performance_metrics(self):
        
        annual_return = self.data['strategy'].iloc[-1] ** (252/len(self.data)) - 1
        daily_return = self.data['daily_return'].mean()
        daily_std = self.data['daily_return'].std()
        sharpe_ratio = daily_return / daily_std * (252**0.5)
        max_drawdown = ((self.data['strategy'].cummax() - self.data['strategy']) / self.data['strategy'].cummax()).max()

        
        return {
            'Annual Return': annual_return,
            'Sharpe Ratio': sharpe_ratio,
            'Max Drawdown': max_drawdown
        }
    def get_rebalance_info(self):
        
        return self.data[self.data['position'].diff().abs() > 0]



If 20 days is selected as the rotation date, the yield is -7%, the Sharpe ratio is 0.23, and the maximum retracement is 97%

In [56]:

strategy = RotationStrategy('沪深300.csv', '中证500.csv')
# 20days
strategy.execute_strategy(20)
# get performance
metrics = strategy.get_performance_metrics()
print(metrics)
# rebalance
rebalance_info = strategy.get_rebalance_info()
rebalance_info.head()

{'Annual Return': -0.07907598865463428, 'Sharpe Ratio': 0.23421015517030963, 'Max Drawdown': 0.9784908300290408}


,Date,Close_hs300,Close_zz500,return_hs300,return_zz500,position,daily_return,strategy
26,2021-12-09,5078.689941,7320.32,0.036753,0.028155,1.0,0.036753,1.243520
30,2021-12-15,5005.899902,7314.26,0.024592,0.029521,0.0,0.029521,1.425485
34,2021-12-21,4913.490234,7275.47,0.000029,0.001940,1.0,0.000029,1.502775
35,2021-12-22,4914.450195,7302.46,-0.000449,0.006388,0.0,0.006388,1.512374
38,2021-12-27,4919.319824,7272.75,0.013996,0.007041,1.0,0.013996,1.568205


If 10 days is selected as the rotation date, the yield is 54%, the Sharpe ratio is 1.11, and the maximum retracement is 85%

In [57]:

strategy2 = RotationStrategy('沪深300.csv', '中证500.csv')
# 10days
strategy2.execute_strategy(10)
# get performance
metrics2 = strategy2.get_performance_metrics()
print(metrics2)
# rebalance
rebalance_info2 = strategy2.get_rebalance_info()
rebalance_info2.head()

{'Annual Return': 0.5415371694234692, 'Sharpe Ratio': 1.1102287635716732, 'Max Drawdown': 0.8507865557567552}


,Date,Close_hs300,Close_zz500,return_hs300,return_zz500,position,daily_return,strategy
11,2021-11-18,4837.620117,7084.68,-0.006392,0.001531,1.0,-0.006392,1.005881
12,2021-11-19,4890.060059,7169.88,0.009853,0.025454,0.0,0.025454,1.031484
25,2021-12-08,4995.930176,7302.99,0.016123,0.006461,1.0,0.016123,1.289571
35,2021-12-22,4914.450195,7302.46,-0.016309,-0.000073,0.0,-0.000073,1.643200
54,2022-01-19,4780.379883,7118.76,-0.018023,-0.014387,1.0,-0.018023,1.323146


The 28 round strategy is based on the CSI 300 and CSI 500 index price trend to determine the timing strategy of large and small round movement. It means that the return rate of stocks has a tendency to continue the original direction of movement, that is, the return rate of stocks with higher returns in the past period of time will still be higher than that of stocks with lower returns in the past. In general, it is a trend investment method designed according to momentum effect. Among them, "two" represents the number of large-cap stocks accounting for about 20% (corresponding to the CSI 300 index), and "eight" represents the number of small-cap stocks accounting for about 80% (corresponding to the CSI 500 index). Twenty-eight rounds of movement refers to the continuous switch between large-cap stocks and small-cap stocks, taking turns to hold: the market performance is strong, then follow the trend to invest in the index; Small and medium cap performance is strong, then follow the trend of investment in small and medium cap index representatives; When the market as a whole is not good, large and small stocks are weak, they hold fixed income assets such as currencies or bonds. Judging indicators: Momentum (closing price of the day - closing price of the previous 20 days)/closing price of the day, judging conditions: when the momentum is greater than 0 and there is no position, buy the larger momentum and hold it, small momentum position adjustment date: closing adjustment of each trading day

Improvements and shortcomings:
1. Instead of using a longer period of time, the return rate of 28 rotation strategies in different periods can be tested according to the regime change of the market
2. The strategy does not consider the actual situation, the cost of fund redemption and position, as well as some micro-order price differences. It is possible to add transaction market processing fees as well as micro-market model assistance strategies
3. It is also necessary to consider the turnover rate, account capital turnover rate, and the yield rate of fixed income products. Add the turnover rate indicator calculation, the liquidity of the client account, and the bond yield considering different times (taking into account the yield on the maturity date)